In [1]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras.models import Sequential,load_model
from keras.layers import Embedding,Dense,Flatten
#from keras.models import load_model
import numpy as np
import os,sys
import math


Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tenso

In [2]:
label_dict_static={}
def create_static_label():
    
    with open("/home/ubuntu/composite.txt") as file:
        for _ in range(18227):
            line  = file.readline().strip().split(",")
            label_dict_static[line[0]] = line[1]
create_static_label()

label_dict_dynamic={}
def create_dynamic_label():
    
    with open("/home/ubuntu/composite_dynamic.txt") as file:
        for _ in range(18227):
            line  = file.readline().strip().split(",")
            label_dict_dynamic[line[0]] = line[1]
create_dynamic_label()


In [3]:
word2vec_model_static = KeyedVectors.load("/home/ubuntu/first.model", mmap='r')
word2vec_model_dynamic = KeyedVectors.load("/home/ubuntu/dynamic_word.model", mmap='r')

In [4]:
import os
#nb_words=len(word2vec_model.wv.vocab), will be parameter for tokenizer
tokenizer = Tokenizer(filters='#\n')
tokenizer.fit_on_texts(word2vec_model_static.wv.vocab.keys())
word_index = tokenizer.word_index
temp_array=[]
validation_size = 50
def get_validation_data(file_names):
    c=0 
    temp_array=[]
    
    while True:
        if c>=len(file_names):
            c=0
        for i,j in enumerate(file_names[c:c+validation_size]):
            l = open("/home/ubuntu/static_validation/"+j,'r').read().strip()
            padded_sequence =  sequence.pad_sequences(tokenizer.texts_to_sequences([l]),maxlen=4000,padding='post',truncating='post')
            temp_array.append(padded_sequence[0])
            

        temp_array = np.array(temp_array)
        #label_array = np.array(label_array)
        #temp_array = temp_array[..., np.newaxis]
        #print("\nYIELDING FROM c = ",c," c+validation_size = ",c+validation_size," and length of temp_array = ",len(temp_array))
        yield (temp_array)
        temp_array=[]
        c+=validation_size
list_of_validation_files = os.listdir("/home/ubuntu/static_validation")
#print(list_of_validation_files)
list_of_validation_files.sort()
validation_generator = get_validation_data(list_of_validation_files)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [5]:
static_model=load_model("/home/ubuntu/static-keras/lstm_model-final-mask_zero.h5")
dynamic_model = load_model("/home/ubuntu/dynamic-keras/final-dynamic_lstm_model-maskZero.h5")




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [6]:
prediction_static = static_model.predict_generator(validation_generator,math.ceil(len(list_of_validation_files)/validation_size))

In [7]:
tokenizer = Tokenizer(filters='#\n')
tokenizer.fit_on_texts(word2vec_model_dynamic.wv.vocab.keys())
word_index = tokenizer.word_index
temp_array=[]
validation_size = 50
def get_validation_data(file_names):
    c=0 
    temp_array=[]
    
    while True:
        if c>=len(file_names):
            c=0
        for i,j in enumerate(file_names[c:c+validation_size]):
            l = open("/home/ubuntu/dynamic_validation/"+j,'r').read().strip()
            padded_sequence =  sequence.pad_sequences(tokenizer.texts_to_sequences([l]),maxlen=4000,padding='post',truncating='post')
            temp_array.append(padded_sequence[0])
            

        temp_array = np.array(temp_array)
        #label_array = np.array(label_array)
        #temp_array = temp_array[..., np.newaxis]
        #print("\nYIELDING FROM c = ",c," c+validation_size = ",c+validation_size," and length of temp_array = ",len(temp_array))
        yield (temp_array)
        
        temp_array=[]
        c+=validation_size
list_of_validation_files = os.listdir("/home/ubuntu/dynamic_validation")
#print(list_of_validation_files)
list_of_validation_files.sort()
validation_generator = get_validation_data(list_of_validation_files)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [8]:
prediction_dynamic = dynamic_model.predict_generator(validation_generator,math.ceil(len(list_of_validation_files)/validation_size))

In [9]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.clf()
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",verticalalignment="top",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('Confusion-Matrix-Hybrid.png')
    plt.clf()


In [10]:
from sklearn.metrics import confusion_matrix
actual_labels=[]
for i,j in enumerate(list_of_validation_files):
    if int(label_dict_dynamic.get(j))==1:
                actual_labels.append(1)
    else:
                actual_labels.append(0)
#print(actual_labels)
#print(prediction)
prediction = (prediction_dynamic+prediction_static)/2
print(prediction[:3])
print(prediction_dynamic[:3])
print(prediction_static[:3])
prediction1 = np.argmax(prediction, axis=1) 
print(prediction1)

cm = confusion_matrix(actual_labels, prediction1)
plot_confusion_matrix(cm,['benign','malicious'],title='CONFUSION MATRIX')

[[0.01085447 0.9891455 ]
 [0.00115068 0.99884933]
 [0.00644273 0.99355733]]
[[2.1644326e-05 9.9997830e-01]
 [4.4914414e-05 9.9995506e-01]
 [2.9856481e-05 9.9997020e-01]]
[[0.02168729 0.97831273]
 [0.00225645 0.9977436 ]
 [0.01285561 0.9871444 ]]
[1 1 1 ... 1 1 0]
Confusion matrix, without normalization
[[1997    3]
 [  24  976]]


<Figure size 432x288 with 0 Axes>

In [11]:

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from funcsigs import signature
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score
from inspect import signature

def call_precision_recall_curve(truelabel,predictedlabel,actualprediction,label):
    
    #print(truelabel.shape)
    #print(actualprediction.shape)
    
    precision, recall, _ = precision_recall_curve(truelabel,actualprediction)
    au = auc(recall, precision)
    print("Area under graph "+str(au))
    print("-------------------------------------------------")
    rec_score = recall_score(truelabel,predictedlabel,pos_label=1,average='binary')
    precise_score = precision_score(truelabel,predictedlabel,pos_label=1,average='binary')
    f1 = f1_score(truelabel,predictedlabel,pos_label=1, average='binary')
    print("Precision for Malicious apps "+str(precise_score))
    print("Recall for Malicious apps "+str(rec_score))
    print("F1-score for Malicious apps  " + str(f1))
    print("-------------------------------------------------")
    rec_score = recall_score(truelabel,predictedlabel,pos_label=0,average='binary')
    precise_score = precision_score(truelabel,predictedlabel,pos_label=0,average='binary')
    f1 = f1_score(truelabel,predictedlabel,pos_label=0, average='binary')
    print("Precision for Benign apps "+str(precise_score))
    print("Recall for Benign apps "+str(rec_score))
    print("F1-score for Benign apps  " + str(f1))
    
    
    #plot the no-skill line too
    positive_cases = sum(truelabel)/len(truelabel)
    plt.plot([0, 1], [positive_cases, positive_cases], linestyle='--')
    
    
    
    # In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve')
    plt.tight_layout()
    plt.savefig('Precision-Recall-Curve-Hybrid.png')
    plt.clf()
prediction_for_1=[]
for each_prediction in prediction:
    prediction_for_1.append(each_prediction[1])
call_precision_recall_curve(np.array(actual_labels),np.array(prediction1),np.array(prediction_for_1),1)

Area under graph 0.9978915902538458
-------------------------------------------------
Precision for Malicious apps 0.9969356486210419
Recall for Malicious apps 0.976
F1-score for Malicious apps  0.9863567458312279
-------------------------------------------------
Precision for Benign apps 0.9881246907471549
Recall for Benign apps 0.9985
F1-score for Benign apps  0.99328525242477


<Figure size 432x288 with 0 Axes>

In [2]:
import tensorflow
tensorflow.__version__

'1.14.0'